In [1]:
import pandas as pd
from thefuzz import process
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
import numpy as np

# Load the first CSV file into a DataFrame
fifa_players = pd.read_csv('male_players.csv')

combined_player_stats = pd.read_csv("filtered_combined_player_stats.csv")

/tmp/ipykernel_2871807/3486642863.py:13: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  fifa_players = pd.read_csv('male_players.csv')


In [2]:
#league_id 
#LaLiga - 53
#Prem - 13
#Bundesliga - 19
#Serie A - 31
#Ligue 1 - 16

filtered_fifa_players = fifa_players[fifa_players['league_id'].isin([53, 13, 19, 31, 16])]

fifa23_players = filtered_fifa_players[filtered_fifa_players['fifa_version'].isin([23])]
fifa23_players.head()
print(fifa23_players.shape[0])

2880


In [3]:
known_names = fifa23_players['long_name']

# Add a new column called 'rating' to the player_stats DataFrame
combined_player_stats['rating'] = None

# Loop through each corrupted name in the player_stats DataFrame
for idx, name in enumerate(combined_player_stats['Player']):
    # Find the best match from the list of known names
    best_match = process.extractOne(name, known_names)
    if best_match is not None:
        matched_name = best_match[0]
        rating = fifa23_players.loc[fifa23_players['long_name'] == matched_name, 'overall'].iloc[0]
        combined_player_stats.at[idx, 'rating'] = rating
        # Additional checks
        squad = combined_player_stats.at[idx, 'Squad']
        born_year = combined_player_stats.at[idx, 'Born']
        # Find the matched player's info
        matched_player = fifa23_players[fifa23_players['long_name'] == matched_name]
        if not matched_player.empty:
            matched_squad = matched_player['club_name'].iloc[0]
            matched_born_year = int(matched_player['dob'].iloc[0].split('-')[0])
            if born_year == matched_born_year:
                combined_player_stats.at[idx, 'rating'] = rating/99
                #print(best_match)
                #print(f"Matched name: {name}, Best match: {matched_name}, Rating: {rating}")
                #print(f"Squad and birth year matched for {name}")
            else:
                combined_player_stats.drop(idx, inplace=True)

In [4]:
# Replace NaN values with 0
combined_player_stats.fillna(0, inplace=True)
#shuffle the players
combined_player_stats = combined_player_stats.sample(frac=1).reset_index(drop=True)

# Split positions into a list
combined_player_stats['Positions_list'] = combined_player_stats['Pos'].str.split(',')

# Get unique positions
unique_positions = set(position for sublist in combined_player_stats['Positions_list'] for position in sublist)

# Create binary indicator variables for each position
for position in unique_positions:
    combined_player_stats[position] = combined_player_stats['Positions_list'].apply(lambda x: 1 if position in x else 0)

# Drop unnecessary columns
combined_player_stats.drop(['Positions_list'], axis=1, inplace=True)

combined_player_stats.head()

# Reorder the columns to make 'rating' the last column
columns_ordered = [col for col in combined_player_stats.columns if col != 'rating'] + ['rating']

# Reassign the DataFrame with the reordered columns
combined_player_stats = combined_player_stats[columns_ordered]

# Display the updated DataFrame
#combined_player_stats.head()

In [5]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

# Extracting features (X) and target variable (Y)
X = combined_player_stats.iloc[1:, 5:-1].values
Y = combined_player_stats.iloc[1:, -1].values  # Only the last column, excluding first row

# Splitting the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Polynomial features
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

# Ridge regression with cross-validation
alpha = 1.0  # Regularization strength
ridge = Ridge(alpha=alpha)

# Perform cross-validation
cv_scores = cross_val_score(ridge, X_train_poly, Y_train, cv=5, scoring='neg_mean_squared_error')
cv_mse = -cv_scores.mean()

# Fit the model
ridge.fit(X_train_poly, Y_train)

# Predictions
Y_pred = ridge.predict(X_test_poly)

# Compute evaluation metrics
mae = mean_absolute_error(Y_test, Y_pred)
mse = mean_squared_error(Y_test, Y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(Y_test, Y_pred)

# Print evaluation metrics
print("Cross-Validation Mean Squared Error (MSE):", cv_mse)
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2):", r2)

Cross-Validation Mean Squared Error (MSE): 0.02519982507362311
Mean Absolute Error (MAE): 0.08557073949911793
Mean Squared Error (MSE): 0.03722976526344689
Root Mean Squared Error (RMSE): 0.19295016264166995
R-squared (R2): -7.063354106404496


In [6]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import uniform

# Extracting features (X) and target variable (Y)
X = combined_player_stats.iloc[1:, 5:-1].values
Y = combined_player_stats.iloc[1:, -1].values  # Only the last column, excluding first row

# Splitting the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Polynomial features
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

# Define the parameter distribution
param_dist = {'alpha': uniform(0.5, 9.5)}  # alpha values from 0.5 to 10

# Ridge regression model
ridge = Ridge()

# Randomized search with cross-validation
random_search = RandomizedSearchCV(estimator=ridge, param_distributions=param_dist, n_iter=20, cv=5, scoring='neg_mean_squared_error', random_state=42)

# Fit RandomizedSearchCV to find the best hyperparameters
random_search.fit(X_train_poly, Y_train)

# Get the best model and its hyperparameters
best_model_ridge = random_search.best_estimator_
best_params_ridge = random_search.best_params_

# Print the best hyperparameters
print("Best Hyperparameters:")
print(best_params_ridge)

# Predictions using the best model
Y_pred_ridge_best = best_model_ridge.predict(X_test_poly)

# Compute evaluation metrics with the best model
mae_ridge_best = mean_absolute_error(Y_test, Y_pred_ridge_best)
mse_ridge_best = mean_squared_error(Y_test, Y_pred_ridge_best)
rmse_ridge_best = np.sqrt(mse_ridge_best)
r2_ridge_best = r2_score(Y_test, Y_pred_ridge_best)

# Print evaluation metrics with the best model
print("\nCross-Validation Mean Squared Error (MSE):", -random_search.best_score_)
print("Mean Absolute Error (MAE):", mae_ridge_best)
print("Mean Squared Error (MSE):", mse_ridge_best)
print("Root Mean Squared Error (RMSE):", rmse_ridge_best)
print("R-squared (R2):", r2_ridge_best)


Best Hyperparameters:
{'alpha': 9.714143595538946}

Cross-Validation Mean Squared Error (MSE): 0.011209685353886303
Mean Absolute Error (MAE): 0.06313956602011805
Mean Squared Error (MSE): 0.014168365560073455
Root Mean Squared Error (RMSE): 0.11903094370823687
R-squared (R2): -2.068634674740388


In [8]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

combined_player_stats['Squad'] = combined_player_stats['Squad'].astype(str)
combined_player_stats['Comp'] = combined_player_stats['Comp'].astype(str)
combined_player_stats['Nation'] = combined_player_stats['Nation'].astype(str)
#combined_player_stats['Pos'] = combined_player_stats['Pos'].astype(str)

# Splitting the data into input (X) and output (Y) variables
X = combined_player_stats.iloc[1:, :-1].drop(columns=['Player','Pos'])
Y = combined_player_stats.iloc[1:, -1]   # Only the last column, excluding first row

# Define categorical columns
categorical_cols = ['Squad', 'Comp', 'Nation']

# Define preprocessing for numerical and categorical features
numeric_features = [col for col in X.columns if col not in categorical_cols]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2))
])
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the polynomial regression model with Ridge regularization
model = Ridge(alpha=10)  # Ridge regression for polynomial regression

# Create a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', model)
])

# Splitting the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Fit the pipeline
pipeline.fit(X_train, Y_train)

# Predictions
Y_pred = pipeline.predict(X_test)

# Compute evaluation metrics
mae = mean_absolute_error(Y_test, Y_pred)
mse = mean_squared_error(Y_test, Y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(Y_test, Y_pred)

# Print evaluation metrics
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2):", r2)

Mean Absolute Error (MAE): 0.06129741110890573
Mean Squared Error (MSE): 0.012367106452142368
Root Mean Squared Error (RMSE): 0.11120749278777203
R-squared (R2): -1.6785116126727675
